# Theoretical velocity profile

In [78]:
# import matplotlib.pyplot as plt
import numpy as np

SEED = 12345

In [79]:
def get_vel_profile(
    Re: float, mu: float, rho: float, L: float, R: float, Nr: int
) -> (np.ndarray, np.ndarray):
    """
    Calculate the velocity profile in a duct.

    Args:
        Re (float): Reynolds number.
        mu (float): Viscosity of the fluid [Pas].
        rho (float): Density of the fluid [kg/m^3].
        L (float): Length of the duct [m].
        R (float): Radius of the duct [m].
        Nr (int): Number of radial points for the profile.

    Returns:
        r (np.ndarray): Array of radial coordinates [m].
        velx_y (np.ndarray): Array of velocities at radial positions [m/s].

    If Re is within the laminar range (Re <= 2000), it computes the velocity profile using
    Hagen-Poiseuille equation.
    If Re is in the transitional range (2000 < Re <= 10000), it returns None.
    If Re is in the turbulent range (10000 < Re <= 10000000), it calculates the velocity
    profile using a power-law model.
    """
    Ks = 2.2e9  # Bulk modulus od water [Pa]
    r = np.linspace(0, R, Nr)
    velx_avg = Re * mu / (rho * 2 * R)  # average velocity [m/s]
    Ma = velx_avg / np.sqrt(Ks / 1000)
    if Re <= 2000 and Ma < 0.8:  # laminar (Hagen-Poiseuille)
        deltap = -8 * mu * velx_avg * L / (R**2)
        velx_y = 1 / 4 * deltap / (mu * L) * (r**2 - R**2)

    elif 2000 < Re <= 10000 and Ma < 0.8:
        velx_y = None
    elif 10000 < Re <= 10000000 and Ma < 0.8:  # turbulent
        n = 2.021 * Re**0.1077  # for Re between 1e4 and 1e6
        velx_max = velx_avg / (
            2 * n**2 / ((n + 1) * (2 * n + 1))
        )  # can be demonstrated for a power law
        velx_y = velx_max * (1 - r / R) ** (1 / n)  # power-law velocity profile
    else:
        velx_y = None
    return r, velx_y

In [80]:
 np.linspace(0, R, Nr)

array([0.        , 0.0024714 , 0.00494281, 0.00741421, 0.00988561,
       0.01235701, 0.01482842, 0.01729982, 0.01977122, 0.02224262,
       0.02471403, 0.02718543, 0.02965683, 0.03212823, 0.03459964,
       0.03707104, 0.03954244, 0.04201384, 0.04448525, 0.04695665,
       0.04942805, 0.05189945, 0.05437086, 0.05684226, 0.05931366,
       0.06178506, 0.06425647, 0.06672787, 0.06919927, 0.07167067,
       0.07414208, 0.07661348, 0.07908488, 0.08155629, 0.08402769,
       0.08649909, 0.08897049, 0.0914419 , 0.0939133 , 0.0963847 ,
       0.0988561 , 0.10132751, 0.10379891, 0.10627031, 0.10874171,
       0.11121312, 0.11368452, 0.11615592, 0.11862732, 0.12109873])

In [81]:
N_profiles = 80000
Nr = 50
rand_gen = np.random.default_rng(seed=SEED)

Re_all = np.zeros((N_profiles, 1))
space_all = np.zeros((N_profiles, Nr))
data_all = np.zeros((N_profiles, Nr + 4))

i = 0
while i < N_profiles:
    # sampling
    if rand_gen.random() < 0.10:
        # laminar
        Re = rand_gen.uniform(1.0, 2e3)  # Reynolds number [-] in interval (1.0, 1e6)
    else:
        # turbulent
        Re = rand_gen.uniform(1e4, 1e6)  # Reynolds number [-] in interval (1.0, 1e6)
    R = rand_gen.uniform(0.05, 0.5)  # Radius of the duct [m] in interval (0.1, .5)
    L = rand_gen.uniform(5.0, 100.0)  # Length of the duct [m] in interval (5.0, 100.0)
    rho = np.abs(
        rand_gen.normal(loc=1000.0, scale=50.0)
    )  # Density of the liquid [kg/m^3] in interval (.0, +inf)
    mu = np.abs(
        rand_gen.normal(loc=1e-3, scale=5e-4)
    )  # Viscosity of the liquid [Pas] in interval (.0, +inf)
    # print(Re, mu, rho, L, R)
    r, velx_y = get_vel_profile(Re, mu, rho, L, R, Nr)
    if velx_y is None:
        # print("Skip Re:", Re)
        continue
    # store
    Re_all[i, :] = Re
    space_all[i, :] = r
    data_all[i, :] = np.concatenate((np.array([mu, rho, L, R]), velx_y))
    # plt.plot(velx_y, r)
    # plt.xlabel('u [m/s]')
    # plt.ylabel('r [m]')
    # plt.xlim([0, 0.0001])
    i += 1

In [82]:
 np.concatenate((np.array([mu, rho, L, R]), velx_y)).shape

(54,)

Save on file

In [83]:
np.savetxt(
    "velprof-data.csv",
    data_all,
    delimiter=",",
    fmt="%12.8e",
    header=",".join(["mu(Pas)", "rho(kg/m3)", "L(m)", "R(m)"]) + ","
    + ",".join([f"vel[{i}](m/s)" for i in range(Nr)]),
    comments="",
)
np.savetxt(
    "velprof-space.csv",
    space_all,
    delimiter=",",
    fmt="%12.8e",
    header=",".join([f"r[{i}](m)" for i in range(Nr)]),
    comments="",
)
np.savetxt(
    "velprof-Re.csv", Re_all, delimiter=",", fmt="%12.8e", header="Re(-)", comments=""
)

In [ ]:
# plt.hist(Re_all, bins=300)